In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [6]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [7]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [8]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [9]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [10]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:  \
           Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   
250    7.214900    0  249  114.20  594  113.71  0.47  0.02  105.48   
2750   2.333243   10  249  112.14  534  111.25  0.89  0.01  104.10   
5250   3.265947   20  249  111.61  519  110.45  1.16  0.01  103.78   
7750   2.846856   30  249  111.30  511  109.95  1.34  0.01  103.60   
10250  2.828463   40  249  111.09  505  109.60  1.48  0.01  103.44   
12750  2.600546   50  249  110.94  501  109.35  1.58  0.01  103.42   
15250  3.012909   60  249  110.81  498  109.14  1.67  0.01  103.27   
17750  2.810593   70  249  110.71  496  108.96  1.74  0.01  103.20   
20250  2.946020   80  249  110.62  493  108.80  1.81  0.01  103.14   
22750  2.975118   90  249  110.54  491  108.67  1.86  0.01  103.12   
25250  3.016478  100  249  110.47  490  108.56  1.91  0.01  103.10   
27750  2.767917  110  249  110.42  488  108.46  1.95  0.00  103.09   
30250  3.935129  120  249  110.36  487  108.37  1.99  0.00  103.07   
32750  3.581231  130  249  110.32  486  108.29  2.02  0.00  103.06   
35250  5.546690  140  249  110.28  485  108.22  2.05  0.00  103.06   
37750  3.836069  150  249  110.24  484  108.16  2.07  0.00  103.06   
40250  3.834341  160  249  110.21  483  108.11  2.10  0.00  103.05   
42750  2.818463  170  249  110.18  482  108.05  2.12  0.00  103.05   
45250  4.677889  180  249  110.15  481  108.01  2.14  0.00  103.04   

                                        TEST:             SPEC:              \
              PPL     NLL    KL   REG    LOSS         PPL     1     2     3   
250    534.625549  105.12  0.35  0.00  103.60  531.430420  0.14  0.15  0.18   
2750   498.482422  102.96  1.14  0.00  102.28  493.127136  0.31  0.56  0.46   
5250   490.206909  102.34  1.44  0.00  102.05  487.723785  0.32  0.59  0.52   
7750   485.525452  101.96  1.64  0.00  101.88  484.019684  0.33  0.61  0.53   
10250  481.135925  101.66  1.78  0.00  101.75  481.263519  0.33  0.61  0.55   
12750  479.858368  101.58  1.84  0.00  101.72  480.323547  0.33  0.65  0.53   
15250  475.836761  101.32  1.95  0.00  101.60  477.388763  0.33  0.67  0.57   
17750  473.799286  101.18  2.02  0.00  101.56  476.354797  0.33  0.68  0.59   
20250  472.299042  101.07  2.07  0.00  101.52  475.167633  0.34  0.68  0.59   
22750  471.714752  101.02  2.10  0.00  101.50  474.584351  0.33  0.68  0.59   
25250  471.189606  100.97  2.13  0.00  101.48  473.864563  0.34  0.68  0.59   
27750  471.029663  100.94  2.15  0.00  101.48  473.488495  0.33  0.68  0.59   
30250  470.663239  100.89  2.18  0.00  101.45  472.877716  0.34  0.68  0.59   
32750  470.496979  100.88  2.18  0.00  101.45  472.637878  0.33  0.68  0.58   
35250  470.443115  100.85  2.21  0.00  101.43  472.066223  0.33  0.68  0.59   
37750  470.429443  100.85  2.21  0.00  101.43  471.856476  0.33  0.68  0.58   
40250  470.439331  100.82  2.23  0.00  101.43  471.856476  0.33  0.68  0.58   
42750  470.250031  100.81  2.23  0.00  101.41  471.171631  0.33  0.68  0.58   
45250  470.252258  100.79  2.26  0.00  101.41  471.171631  0.33  0.68  0.58   

      HIER:        
      CHILD OTHER  
250    0.87  0.75  
2750   0.30  0.23  
5250   0.31  0.19  
7750   0.27  0.13  
10250  0.26  0.14  
12750  0.26  0.14  
15250  0.22  0.11  
17750  0.19  0.10  
20250  0.18  0.09  
22750  0.17  0.09  
25250  0.17  0.09  
27750  0.17  0.09  
30250  0.17  0.09  
32750  0.17  0.09  
35250  0.18  0.09  
37750  0.17  0.09  
40250  0.18  0.09  
42750  0.17  0.09  
45250  0.18  0.09

0 R: 1.000 P: 0.376 carry pockets strap back nice work quality room lot space
   1 R: 0.225 P: 0.106 pocket room mouse power ipad charger small netbook inch perfectly
     11 R: 0.060 P: 0.060 ! love perfect color recommend ... perfectly loves bought absolutely
     14 R: 0.026 P: 0.026 price amazon quality bought $ inch buy found ... reviews
     12 R: 0.033 P: 0.033 ! shipping received return item amazon ordered arrived ... seller
   2 R: 0.189 P: 0.088 sleeve protection inside zipper padding nice air pro bit inch
     23 R: 0.060 P: 0.060 mac pro cover perfectly apple protect hard shell scratches air
     21 R: 0.041 P: 0.041 smell $ quality cheap price made bought worth bad buy
   3 R: 0.079 P: 0.028 ; & pro air perfectly made 'm ... size price
     31 R: 0.051 P: 0.051 color cover keyboard love blue picture nice screen pink protector
   5 R: 0.038 P: 0.013 - : zipper strap handle flap pros cons top bottom
     51 R: 0.024 P: 0.024 broke months started weeks zipper month year week 